# BVD Model Fit for Admittance Data
This notebook replicates the `ksq_fit2.py` workflow with logical cells and improved subplot usage.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
import numpy as np
import matplotlib.patches as patches
from scipy.optimize import least_squares

plt.close('all')


In [ ]:
filename = 'S0_lambda10.xlsx'
df = pd.read_excel(filename)
f = df['freq (Hz)'].values
df['Admittance (S)'] = df['Admittance (S)'].str.replace('i', 'j')
Ydata = df['Admittance (S)'].astype('complex128').values


In [ ]:
fs = 625094243
C0_guess = 8.85e-11
k2_guess = 0.006
Cm_guess = k2_guess * C0_guess
Lm_guess = 1 / ((2 * np.pi * fs) ** 2 * Cm_guess)
Q = 3000
Rm_guess = np.sqrt(Lm_guess / Cm_guess) / Q
w = 2 * np.pi * f
scales = np.array([1e-11, 1e-13, 1e-7, 1.0])

p0 = np.array([C0_guess, Cm_guess, Lm_guess, Rm_guess])
p0_hat = p0 / scales


In [ ]:
def Y_bvd(p_hat, w):
    C0, Cm, Lm, Rm = p_hat * scales
    Zm = Rm + 1j * (w * Lm - 1 / (w * Cm))
    return 1j * w * C0 + 1 / Zm

def residual(p_hat, w, Y):
    res = Y_bvd(p_hat, w) - Y
    return np.concatenate((res.real, res.imag))

def jacobian(p_hat, w, Y):
    C0, Cm, Lm, Rm = p_hat * scales
    Zm = Rm + 1j * (w * Lm - 1 / (w * Cm))
    inv_Zm = 1 / Zm
    inv_Zm2 = inv_Zm ** 2

    dC0 = 1j * w
    dRm = -inv_Zm2
    dLm = -1j * w * inv_Zm2
    dCm = -1j / (w * Cm ** 2) * inv_Zm2

    grads = np.vstack((dC0, dCm, dLm, dRm)).T * scales
    J = np.vstack((grads.real, grads.imag))
    return J


In [ ]:
sol = least_squares(
    residual,
    p0_hat,
    args=(w, Ydata),
    x_scale='jac',
    diff_step=1e-3,
    max_nfev=2000,
    jac=jacobian,
)

p_opt = sol.x * scales
C0, Cm, Lm, Rm = p_opt
k2_eff = Cm / (C0 + Cm)
Qm = np.sqrt(Lm / Cm) / Rm

print(f'k_eff^2 = {k2_eff:.4f},   Q_m = {Qm:.0f}')
print('Guess:', p0)
print('Scaled Solution:', sol.x)
print('Delta (scaled):', sol.x - p0_hat)

fp = np.sqrt(fs ** 2 / (1 - k2_guess))
fstp = (fp - fs) / 333
f_dense = np.linspace(fs - 333 * fstp, fp + 333 * fstp, 1000)
w_dense = 2 * np.pi * f_dense
Yfit = Y_bvd(p_opt / scales, w_dense)
Yguess = Y_bvd(p0_hat, w_dense)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(f / 1e6, 20 * np.log10(np.abs(Ydata)), label='Measured', linestyle='none', marker='.', markevery=26, markersize=10)
ax.plot(f_dense / 1e6, 20 * np.log10(np.abs(Yfit)), label='Fitted', linestyle='-')
ax.plot(f_dense / 1e6, 20 * np.log10(np.abs(Yguess)), label='Estimate', linestyle='--')
ax.set_xlabel('Frequency (MHz)', fontsize='large')
ax.set_ylabel('Admittance [dB]', fontsize='large')
ax.set_title('BVD Model Fit vs Measured Admittance')
ax.legend(fontsize='large')
ax.grid(True)
ax.annotate('Fit Results:', xy=(0.2, 0.45), xycoords='figure fraction', fontsize=14)
ax.annotate(r'$C_0 = $' + f"{C0 * 1e12:.2f} pF", xy=(0.2, 0.40), xycoords='figure fraction', fontsize=14)
ax.annotate(r'$C_m = $' + f"{Cm * 1e12:.2f} pF", xy=(0.2, 0.35), xycoords='figure fraction', fontsize=14)
ax.annotate(r'$L_m = $' + f"{Lm * 1e9:.2f} nH", xy=(0.2, 0.30), xycoords='figure fraction', fontsize=14)
ax.annotate(r'$R_m = $' + f"{Rm:.3f} \u03a9", xy=(0.2, 0.25), xycoords='figure fraction', fontsize=14)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(f, np.real(Ydata), label='Measured')
ax.plot(f_dense, np.real(Yfit), label='Fitted', linestyle='--')
ax.plot(f_dense, np.real(Yguess), label='Estimate', linestyle=':')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Real(Admittance)')
ax.set_title('Real Part Comparison')
ax.legend()
ax.grid(True)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(f / 1e6, np.imag(Ydata), label='Measured')
ax.plot(f_dense / 1e6, np.imag(Yfit), label='Fitted', linestyle='--')
ax.plot(f_dense / 1e6, np.imag(Yguess), label='Estimate', linestyle=':')
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Imag(Admittance)')
ax.set_title('Imaginary Part Comparison')
ax.legend()
ax.grid(True)
plt.show()
